![LogoUC3M](https://www.fundacion.uc3m.es/wp-content/uploads/2018/11/Logo-UC3M-nuevo.png)

### Aprendizaje Automático · Grado en Ingeniería Informática · Curso 2022/23
# Tutorial 2: Hyperparameter Optimization with Decision Trees

We will install the library `scikit-optimize`, which contains functions that allow and ease for hyperparameter optimization.

In [ ]:
!pip install git+https://github.com/scikit-optimize/scikit-optimize.git

# Tuning Decision Trees



We will optimize the following hyperparameters:

- `max_depth`: the maximum depth of the tree. If `None`, then nodes are expanded until all leaves are pure or until all leaves contain less than `min_samples_split` samples. Ignored if `max_leaf_nodes` is not `None`.
    
- `min_samples_split`: the minimum number of samples required to split an internal node.

There are more hyper-parameters which can be considered: 
  - [Documentation for a decision tree classifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
  - [Documentation for a decision tree regressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)



We will work with the California housing dataset. We load the data as usual, with `X` being the attributes and `y` the output.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_california_housing
from scipy.stats import sem

housing = fetch_california_housing()
X = housing.data
y = housing.target

The combination of model evaluation and hyper-parameter tuning can be understood as an external loop (outer) that trains a model and tests the model, and an internal loop (inner), where the training process consists on looking for the best hyper-parameters, and then obtaining the model with those best hyper-parameters.

First, we are going to use **holdout** (train/test) for model evaluation (external loop or **outer**), and **3-fold crossvalidation** for hyper-parameter tuning (internal loop or **inner**). Hyper-parameters will be adjusted using a search procedure.

# Grid Search

Let's define a function for computing the RMSE (root mean squared error).

In [ ]:
def rmse (y_test, y_test_pred):
  """ Computation of root mean squared error """
  return np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))

In [ ]:
from sklearn.model_selection import train_test_split

# Holdout for model evaluation. 33% of available data for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

First, let's compute the error with default hyper-parameters. This can be used as a baseline.

In [ ]:
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor

regr = DecisionTreeRegressor()
np.random.seed(42)
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
print(f"RMSE of tree with default hyper-params: {rmse(y_test, y_pred):.2f}")


We will now use Grid Search to find the best combination of hyper-parameters. To do so, we need to define a grid with the values that we want to test for each parameter.

Grid Search will test all combinations, therefore holds a problem of "combinatorial explosion" when it comes to efficiency.

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import KFold

# Search space
param_grid = {
    'max_depth': range(2, 16, 2),
    'min_samples_split': range(2, 16, 2)
}

# 3-Fold cross validation for the inner loop.
inner = KFold(n_splits=3, shuffle=True, random_state=42)

# Definition of a 2-step process that self-adjusts 2 hyperparams
regr = GridSearchCV(DecisionTreeRegressor(random_state=42), 
                    param_grid,
                    scoring='neg_mean_squared_error',
                    cv=inner, 
                    n_jobs=1,
                    verbose=1)

# Train the self-adjusting process
regr.fit(X_train, y_train)

# At this point, regr contains the model with the best hyper-parameters 
# found by Grid Search and trained on the complete X_train.

In [ ]:
# Now, the performance of regr is computed on the test partition
y_pred = regr.predict(X_test)
print(f"RMSE of tree with hyper-parameter tuning (grid search): {rmse(y_test, y_pred):.2f}")

Let's now check the best hyper-parameters found and their score (MSE). We can see that they are near the extremes of parameter space.

In [ ]:
regr.best_params_, -regr.best_score_

# Randomized Search

Now, let's use **Randomized Search** instead of Grid Search. The approach is similar, but only a random sample of the combinations are tested.

In this particular example, only 10 combinations of hyper-parameter values will be tried (`n_iter=10`).

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn import metrics

# Search space
param_grid = {
    'max_depth': range(2, 16, 2),
    'min_samples_split': range(2, 16, 2)
}

# 3-Fold cross validation for the inner loop.
inner = KFold(n_splits=3, shuffle=True, random_state=42)

# Definition of a 2-step process that self-adjusts 2 hyperparams
regr = RandomizedSearchCV(DecisionTreeRegressor(random_state=42), 
                          param_grid,
                          scoring='neg_mean_squared_error',
                          cv=inner, 
                          n_jobs=1,
                          verbose=1,
                          n_iter=10,
                          random_state=42)

# Train the self-adjusting process
regr.fit(X_train, y_train)

In [ ]:
# Now, the performance of regr is computed on the test partition
y_pred = regr.predict(X_test)
print(f"RMSE of tree with hyper-parameter tuning (randomized search): {rmse(y_test, y_pred):.2f}")

In [ ]:
regr.best_params_, -regr.best_score_

For **Randomized Search**, we can define the search space with statistical distributions, rather than using particular values as we did before. Below you can see how to use a uniform distribution on integers between 2 and 16 by means of *randint*. For continuous hyper-parameters we could use continuous distributions such as *uniform* or *expon* (exponential).

In [ ]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn import metrics

# Search space with integer uniform distributions
param_grid = {
    'max_depth': sp_randint(2, 16),
    'min_samples_split': sp_randint(2, 16)
}

inner = KFold(n_splits=3, shuffle=True, random_state=42)
regr = RandomizedSearchCV(DecisionTreeRegressor(random_state=42), 
                          param_grid,
                          scoring='neg_mean_squared_error',
                          cv=inner, 
                          n_jobs=1,
                          verbose=1,
                          n_iter=10,
                          random_state=42)

regr.fit(X_train, y_train)

In [ ]:
# Now, the performance of regr is computed on the test partition
y_pred = regr.predict(X_test)
print(f"RMSE of tree with hyper-parameter tuning (randomized search): {rmse(y_test, y_pred):.2f}")

In [ ]:
regr.best_params_, -regr.best_score_

What if we wanted to do **model evaluation with 5-fold cross validation** and **hyper-parameter tuning with 3-fold cross validation**? 

This is called [nested cross validation](https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html). There is an external loop (for evaluating models) and an internal loop (for hyper-parameter tuning).

In [ ]:
from scipy.stats import uniform, expon
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn import metrics

# Evaluation of model (outer loop)
outer = KFold(n_splits=5, shuffle=True, random_state=42)

# Search space
param_grid = {'max_depth': list(range(2, 16, 2)),
              'min_samples_split': list(range(2, 16, 2))}

inner = KFold(n_splits=3, shuffle=True, random_state=42)

# This is the internal 3-fold crossvalidation for hyper-parameter tuning
regr = RandomizedSearchCV(DecisionTreeRegressor(random_state=42), 
                          param_grid,
                          scoring='neg_mean_squared_error',
                          # 3-fold for hyper-parameter tuning
                          cv=inner, 
                          n_jobs=1, 
                          verbose=1,
                          n_iter=10,
                          random_state=42)

In [ ]:
# This is the external 5-fold crossvalidation for model evaluation
# Notice that regr is the model resulting of hyper-parameter tuning
# For sklearn, higher scores are better. Given that MSE is an error (smaller is better), the corresponding score is -MSE
scores = -cross_val_score(regr, 
                          X, 
                          y, 
                          scoring='neg_mean_squared_error', 
                          cv=outer)
print(scores)

# The score was MSE, we want RMSE
scores = np.sqrt(scores)

# The mean of the 5-fold crossvalidation is the final score of the model
print(f"{scores.mean()} +- {scores.std()}")

# Obtaining the final model (for deployment, sending to a competition...)

If we need a final model, we can get it by fitting the regresor to all the available data. Let us remember that it does hyper-parameter tuning.

In [ ]:
# Fitting again the randomized search
regrFinal = regr.fit(X, y)

In [ ]:
regr.best_params_, -regr.best_score_

# Bayesian Optimization

In Bayesian optimization, the search is directed towards those most-promising solutions, instead of just searching randomly.

The `scikit-optimize`library will be used for this approach. We will use **holdout** for model evaluation and **3-fold cross validation** for hyper-parameter tuning.

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Integer
from skopt.plots import plot_objective, plot_histogram, plot_convergence
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

# Search space with integer uniform distributions
param_grid = {
    'max_depth': Integer(2, 16),
    'min_samples_split': Integer(2, 16)
}

inner = KFold(n_splits=3, shuffle=True, random_state=42)
regr = BayesSearchCV(DecisionTreeRegressor(random_state=42), 
                     param_grid,
                     scoring='neg_mean_squared_error',
                     cv=inner,    
                     n_jobs=1, 
                     verbose=1,
                     n_iter=20,
                     random_state=42)

regr.fit(X_train, y_train)

# At this point, regr contains the model with the best hyper-parameters 
# found by Bayes search and trained on the complete X_train

In [ ]:
# Now, the performance of regr is computed on the test partition
y_pred = regr.predict(X_test)
print(f"RMSE of tree with hyper-parameter tuning (Bayes search): {rmse(y_test, y_pred):.2f}")

In [ ]:
regr.best_params_, -regr.best_score_

We can check if the optimization has converged:

In [ ]:
_ = plot_convergence(regr.optimizer_results_[0])
plt.show()

In [ ]:
_ = plot_objective(regr.optimizer_results_[0],
                   dimensions=['max_depth', 'min_samples_split'],
                   n_minimum_search=int(1e8))
plt.show()

We can try another alternative with different models:

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

pipe = Pipeline([
    ('model', None)
])

# Search space for DT
param_grid1 = {
    'model': [DecisionTreeRegressor(random_state=42)],
    'model__max_depth': Integer(2, 16),
    'model__min_samples_split': Integer(2, 16)
}

# Search space for KNN
param_grid2 = {
    'model': [KNeighborsRegressor()],
    'model__n_neighbors': Integer(2, 10)
}

# List of the two spaces
param_grid = [param_grid1, param_grid2]

inner = KFold(n_splits=3, shuffle=True, random_state=42)

regr = BayesSearchCV(pipe, 
                     param_grid,
                     scoring='neg_mean_squared_error',
                     cv=inner,    
                     n_jobs=1, verbose=1,
                     n_iter=20,
                     random_state=42)

regr.fit(X=X_train, y=y_train)

In [ ]:
# Now, the performance of regr is computed on the test partition
y_pred = regr.predict(X_test)
print(f"RMSE of tree with hyper-parameter tuning (Bayes search): {rmse(y_test, y_pred):.2f}")

In [ ]:
regr.best_params_, -regr.best_score_